In [1]:
from databases import BDS,Crm,BDSMem,PosicaoDm1, Boletador
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from funcoes_datas import FuncoesDatas
import pandas as pd
from emailer import Email, EmailLer
from pretty_html_table import build_table
import re

[base_dados.py directory]: C:\Temp\Foundation
[databases.py directory]: C:\Temp\Foundation


In [53]:
##Puxa as classes do databases (consultas nas bases sql da empresa)
crm = Crm()
crm_fundos = Crm(load_cadfundos=True)
bds = BDS()
fdt = FuncoesDatas()
bol = Boletador()
posicao = PosicaoDm1

#retorna posição de todas as carteiras + fundos
posicao = PosicaoDm1()

#Retorna o publico alvo dos fundos
publico_alvo = crm. publico_alvo_fundo().rename(columns={'new_cnpj':'CNPJ'})
publico_alvo['CNPJ'] = publico_alvo['CNPJ'].astype('str')
publico_alvo['CNPJ']  = publico_alvo['CNPJ'].apply(lambda x : re.sub('[^0-9]', '',x))

#Retorna a PO_Cadastro
cadastro= posicao.po_cadastro()[['GuidContaCRM','CNPJ']]
cadastro['CNPJ'] = cadastro['CNPJ'].astype('str')

#Fundos exclusivos que possuem o Gold Premium
posicao_fundos = posicao.posicao_fundos_all(hoje, apenas_exclusivos=False)
gold_premium = posicao_fundos[posicao_fundos['GuidProduto']=='d037e458-0a0d-e011-a374-d8d385b9752e']
gold_premium = gold_premium[['GuidContaCRM','NomeContaCRM','FinanceiroFinal']]

#Fazer o merge

gold_premium = pd.merge(left=gold_premium,right=cadastro,on='GuidContaCRM',how='left')

#LInkar com o publico alvo

gold_premium = pd.merge(left=gold_premium,right=publico_alvo,on='CNPJ',how='left')

#PL do Gold
PL = 167403907.55

In [62]:
gold_premium= gold_premium[['NomeContaCRM','FinanceiroFinal','new_publicoalvoname']]
gold_premium['%'] = gold_premium['FinanceiroFinal'] / PL
verificar_qualificados = gold_premium.groupby(['new_publicoalvoname']).sum().reset_index()

In [38]:
b = '01.753.087/0001-30'
re.sub('[^0-9]', '',b)

'01753087000130'